In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from truffletopia import Truffle, UnitOperation

# Create Data

In [20]:
np.random.seed(42)

# read the operations data (describes distributions used to synthesize rates)
df = pd.read_csv('../truffletopia/data/operations.csv')
display(df.head())

# create feed costs
df['Cost (USD/KG)'] = np.random.random(size=101)
display(df.head())

# read in truffle_rate data
oee = pd.read_csv("data/truffle_line_rates.csv")

# drop values less than 0
print('rates less than 0: {} of {}'.format((oee['rate'] < 0).sum(), oee.shape[0]))
oee = oee.loc[oee['rate'] > 0]
oee.reset_index(drop=True, inplace=True)

# change rate to have units
oee.columns = ['base_cake', 'truffle_type', 'primary_flavor', 'secondary_flavor',
       'color_group', 'line', 'rate_kg_per_hr']

# use the product descriptors base_cake, truffle_type, and color_group
# to influence the cost of the product. pd.merge to merge on the matching
# values for each descriptor. add some noise so that the costs are not all
# the same
oee['cost_usd_per_kg'] =\
pd.merge(df, oee, left_on='Classification', right_on='base_cake')['Cost (USD/KG)'] +\
pd.merge(df, oee, left_on='Classification', right_on='truffle_type')['Cost (USD/KG)'] +\
pd.merge(df, oee, left_on='Classification', right_on='color_group')['Cost (USD/KG)'] * np.random.random(size=oee.shape[0])

# create a product name column
x = oee['base_cake'].str[:3]
for col in oee.columns[1:-2]:
    x += oee[col].str[:3]
oee['product'] = x

# the price is set for each unique product
price = np.random.random(size=len(x.unique())) + 1.5
price = pd.Series(price, index=x.unique(), name='price_usd_per_kg')

# merge the prices with the original oee df
oee = pd.merge(oee, price, left_on='product', right_on=price.index)

oee = oee[['product', 'base_cake', 'truffle_type', 'primary_flavor', 'secondary_flavor',
       'color_group', 'line', 'rate_kg_per_hr', 'cost_usd_per_kg', 
       'price_usd_per_kg']]

# ebit is price - cost
oee['ebit_per_kg'] = oee['price_usd_per_kg'] - oee['cost_usd_per_kg']

# ebit/hr is ebit/kg * kg/hr
oee['ebit_per_hr'] = oee['ebit_per_kg'] * oee['rate_kg_per_hr']

# sort by base_cake for dashboard viewing purpoes
col = 'base_cake'
oee = oee.sort_values([col, 'ebit_per_hr'], ascending=False).reset_index(drop=True)

# add kg produced
# random hr * kg/hr
oee['kg_produced'] = np.abs(np.random.normal(loc=1000, scale=500) * oee['rate_kg_per_hr'])

display(oee.head())
oee.to_csv('oee_line.csv', index=False)

# sample 1000 pts
oee_sample = oee.sample(n=1000)

# and remove columns not necessary for dashboard
oee_sample = oee[['base_cake', 'truffle_type', 'primary_flavor', 'secondary_flavor', 'color_group', 'line',
                  'rate_kg_per_hr', 'ebit_per_hr']]
oee_sample.to_csv('oee_line_1000.csv', index=False)

display(oee_sample.head())

,Category,Classification,Mean,Std,Distribution
0,base_cake,Tiramisu,0.230,2.564,normal
1,base_cake,Sponge,0.794,1.064,normal
2,base_cake,Pound,0.141,1.526,normal
3,base_cake,Chiffon,0.047,1.459,normal
4,base_cake,Cheese,0.361,1.483,normal


,Category,Classification,Mean,Std,Distribution,Cost (USD/KG)
0,base_cake,Tiramisu,0.230,2.564,normal,0.374540
1,base_cake,Sponge,0.794,1.064,normal,0.950714
2,base_cake,Pound,0.141,1.526,normal,0.731994
3,base_cake,Chiffon,0.047,1.459,normal,0.598658
4,base_cake,Cheese,0.361,1.483,normal,0.156019


rates less than 0: 625 of 10000


,product,base_cake,truffle_type,primary_flavor,secondary_flavor,color_group,line,rate_kg_per_hr,cost_usd_per_kg,price_usd_per_kg,ebit_per_kg,ebit_per_hr,kg_produced
0,TirChoOraButTeaQ,Tiramisu,Chocolate Outer,Orange Brandy,Butter Rum,Teal,Q,1.825253,0.548046,1.863324,1.315278,2.400716,2712.731890
1,TirCanHorBlaOpaQ,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.581840,0.768758,2.190701,1.421943,2.249287,2350.965848
2,TirCanHorBlaOpaQ,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.525405,0.788353,2.190701,1.402348,2.139148,2267.089864
3,TirCanHorBlaOpaQ,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.493528,0.820693,2.190701,1.370008,2.046145,2219.714528
4,TirJelSouPepTifQ,Tiramisu,Jelly Filled,Sour,Peppermint,Tiffany,Q,1.190711,0.768010,2.390995,1.622986,1.932508,1769.661617


,base_cake,truffle_type,primary_flavor,secondary_flavor,color_group,line,rate_kg_per_hr,ebit_per_hr
0,Tiramisu,Chocolate Outer,Orange Brandy,Butter Rum,Teal,Q,1.825253,2.400716
1,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.581840,2.249287
2,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.525405,2.139148
3,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.493528,2.046145
4,Tiramisu,Jelly Filled,Sour,Peppermint,Tiffany,Q,1.190711,1.932508


# Dashboard Components

In [19]:
# our standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from ipywidgets import interact

# our stats libraries
import random
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy

# our scikit-Learn library for the regression models
import sklearn         
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# our dash libraries
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html, callback_context, Dash
import dash_daq as daq
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

In [24]:
oee = pd.read_csv("oee_line_1000.csv")
oee.head()

,base_cake,truffle_type,primary_flavor,secondary_flavor,color_group,line,rate_kg_per_hr,ebit_per_hr
0,Tiramisu,Chocolate Outer,Orange Brandy,Butter Rum,Teal,Q,1.825253,2.400716
1,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.581840,2.249287
2,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.525405,2.139148
3,Tiramisu,Candy Outer,Horchata,Black Currant,Opal,Q,1.493528,2.046145
4,Tiramisu,Jelly Filled,Sour,Peppermint,Tiffany,Q,1.190711,1.932508


In [29]:
groupby = ['line', 'base_cake', 'truffle_type']
target = 'rate_kg_per_hr'
oee.groupby(groupby)[target].describe()[['50%', '75%']]

50%       75%
line base_cake truffle_type                       
Q    Butter    Candy Outer      0.189936  0.265272
               Jelly Filled     0.164476  0.251915
     Cheese    Candy Outer      0.401260  0.482485
               Chocolate Outer  0.398286  0.492960
     Chiffon   Candy Outer      0.192734  0.280234
               Chocolate Outer  0.185667  0.302140
               Jelly Filled     0.184759  0.254344
     Pound     Candy Outer      0.230861  0.314178
               Chocolate Outer  0.248413  0.332926
               Jelly Filled     0.211891  0.300555
     Sponge    Candy Outer      0.794666  0.854908
               Chocolate Outer  0.767024  0.858616
               Jelly Filled     0.716377  0.798703
     Tiramisu  Candy Outer      0.388201  0.516984
               Chocolate Outer  0.358398  0.490114
               Jelly Filled     0.256767  0.407807
R    Butter    Candy Outer      0.165924  0.257368
               Chocolate Outer  0.185993  0.293564
     Cheese    Candy Outer      0.457534  0.617803
               Chocolate Outer  0.413315  0.530903
               Jelly Filled     0.244406  0.393207
     Chiffon   Candy Outer      0.176516  0.293761
               Chocolate Outer  0.150901  0.251135
               Jelly Filled     0.128518  0.238219
S    Butter    Jelly Filled     0.085826  0.198795
     Cheese    Jelly Filled     0.236473  0.340568
     Sponge    Jelly Filled     0.432875  0.539146
T    Butter    Jelly Filled     0.185793  0.287891
     Chiffon   Jelly Filled     0.211450  0.442287
     Pound     Jelly Filled     0.301471  0.542771
     Sponge    Jelly Filled     0.536847  0.731357

In [30]:
oee.groupby(groupby)[target].agg(['sum', 'std', 'mean'])

sum       std      mean
line base_cake truffle_type                                   
Q    Butter    Candy Outer      100.708506  0.107371  0.207646
               Jelly Filled      20.023599  0.110938  0.200236
     Cheese    Candy Outer      124.706510  0.133001  0.415688
               Chocolate Outer  126.314851  0.133327  0.422458
     Chiffon   Candy Outer       67.963784  0.125266  0.226546
               Chocolate Outer   48.745172  0.142010  0.243726
               Jelly Filled      21.178445  0.132663  0.211784
     Pound     Candy Outer      170.085879  0.135686  0.255768
               Chocolate Outer  112.984393  0.149435  0.282461
               Jelly Filled      78.656255  0.145970  0.236205
     Sponge    Candy Outer      242.824782  0.104089  0.809416
               Chocolate Outer  782.551437  0.132105  0.782551
               Jelly Filled      72.451293  0.118678  0.724513
     Tiramisu  Candy Outer      205.272693  0.235922  0.424995
               Chocolate Outer  270.451462  0.202008  0.395976
               Jelly Filled     118.625611  0.212771  0.316335
R    Butter    Candy Outer       97.024399  0.150901  0.204693
               Chocolate Outer   25.014144  0.173998  0.250141
     Cheese    Candy Outer      257.168579  0.208248  0.515368
               Chocolate Outer  128.108850  0.192801  0.432800
               Jelly Filled      47.216403  0.168551  0.274514
     Chiffon   Candy Outer       31.213588  0.178086  0.231212
               Chocolate Outer   46.573211  0.131268  0.176414
               Jelly Filled      17.885110  0.214717  0.194403
S    Butter    Jelly Filled      13.722266  0.132965  0.138609
     Cheese    Jelly Filled      28.559285  0.171066  0.285593
     Sponge    Jelly Filled      97.080336  0.148382  0.485402
T    Butter    Jelly Filled      32.704694  0.258550  0.249654
     Chiffon   Jelly Filled      87.159337  0.274951  0.299517
     Pound     Jelly Filled     121.455737  0.319703  0.404852
     Sponge    Jelly Filled      54.528196  0.243672  0.550790

In [ ]:
def find_opportunity(df,
                     groupby_primary = "Cost Center",
                     groupby_secondary = "Technology",
                     groupby_tertiary = "Tank Number",
                     time_column=time_components[3],
                     volume_column='Parent Batch Actual Qty',
                     quant_target=0.75):
    groups=3
    if type(df.iloc[-1][time_column]) == pd._libs.tslibs.timedeltas.Timedelta:
        df[time_column] = df[time_column]
    margin_column = "{} By {}".format(volume_column, time_column)
    groupby = [groupby_primary, groupby_secondary, groupby_tertiary]
    df.loc[:, margin_column] = df[volume_column] / df[time_column]
    df = df.loc[df[[margin_column, volume_column, time_column]].notnull().all(axis=1)]
    desc = df.groupby(groupby)[margin_column].describe()[['50%', '75%']]
    totals = df.groupby(groupby)[volume_column, time_column].agg(['sum', 'std', 'mean'])
    count = df.groupby(groupby)[volume_column].agg(['count'])
    count.columns = ['Count']
    desc = desc.join(totals).dropna()
    desc = desc.join(count)
    desc['Volume Opportunity, Gal'] = (desc[volume_column, 'sum'] / desc['50%'] * desc['75%']) - desc[volume_column, 'sum']
    desc['Time Opportunity, Hours'] = desc[time_column, 'sum'] - (desc[time_column, 'sum'] / desc['75%'] * desc['50%'])
    desc = desc.sort_values(by=[('Time Opportunity, Hours')], ascending=False)
    return desc